In [1]:
import pandas as pd
from SleepClasses.AggregatedData import AggregatedData
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import shap
import plotly.express as px
import plotly.graph_objects as go

# Mostra tutte le colonne
pd.set_option('display.max_columns', None)

/Users/boe/Desktop/Unimi/Tesi/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sleep_dfs_agg = pd.read_pickle(r'SleepData/sleep_dfs_agg.pkl')
sleep_dfs_gran = pd.read_pickle(r'SleepData/sleep_dfs_gran.pkl')
agg = AggregatedData(sleep_dfs_gran['2'], sleep_dfs_agg['2'])

df = agg.getAggregatedDataDf()

/Users/boe/Desktop/Unimi/Tesi/code/SleepClasses/AggregatedData.py:594: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(merged_aggregated.values(), ignore_index=True)
/Users/boe/Desktop/Unimi/Tesi/code/SleepClasses/AggregatedData.py:647: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modified_rows_df = pd.concat([modified_rows_df, df.loc[[idx]]])
/Users/boe/Desktop/Unimi/Tesi/code/SleepClasses/AggregatedData.py:650: FutureWarning: The behavior of DataFrame concatenation with 

In [3]:
# Supponiamo che `df` sia il tuo DataFrame
measures = ["HrAverage", "HrMax", "HrMin", "RmssdAverage",
            "RmssdMax", "RmssdMin", "RrAverage", "RrMax", "RrMin", "Sdnn_1Average",
            "Sdnn_1Max", "Sdnn_1Min", "SnoringAverage", "SnoringMax", "SnoringMin",
            "LightSleepDuration", "RemSleepDuration", "DeepSleepDuration",
            "WakeUpDuration", "WakeUpCount", "ManualSleepDuration",
            "UndefinedSleepDuration", "OutOfBedCount", "OutOfBedTime",
            "TotalSleepTime"]

In [4]:
X = df[measures]

In [5]:
model = IsolationForest(n_estimators=100, contamination=0.1)

In [6]:
model.fit(X)

IsolationForest(contamination=0.1)

In [7]:
y_pred = model.predict(X)
X['anomaly'] = y_pred
X[y_pred == -1] # anomalies

/var/folders/tf/6669z1mn2pv1y3dv4rdzn7200000gn/T/ipykernel_25383/3262471593.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['anomaly'] = y_pred


,HrAverage,HrMax,HrMin,RmssdAverage,RmssdMax,RmssdMin,RrAverage,RrMax,RrMin,Sdnn_1Average,Sdnn_1Max,Sdnn_1Min,SnoringAverage,SnoringMax,SnoringMin,LightSleepDuration,RemSleepDuration,DeepSleepDuration,WakeUpDuration,WakeUpCount,ManualSleepDuration,UndefinedSleepDuration,OutOfBedCount,OutOfBedTime,TotalSleepTime,anomaly
0,56,95,47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21180,0,14100,2760,4,0,0,0,0.0,35280.0,-1
3,61,73,51,45.0,145.0,0.0,16.0,23.0,9.0,49.0,54.0,0.0,0.0,0.0,0.0,4500,2520,2160,2640,1,0,0,0,0.0,9180.0,-1
18,66,88,46,28.0,48.0,0.0,15.0,24.0,9.0,44.0,50.0,0.0,1.0,100.0,0.0,21660,3420,3420,6780,6,0,0,3,6960.0,28500.0,-1
27,64,91,46,34.0,59.0,0.0,16.0,25.0,8.0,49.0,55.0,0.0,0.0,0.0,0.0,23820,3420,4260,10020,5,0,0,3,3300.0,31500.0,-1
41,60,80,50,42.0,45.0,32.0,16.0,22.0,8.0,53.0,56.0,42.0,1.0,100.0,0.0,18840,5220,2040,5040,3,0,0,2,1140.0,26100.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,62,88,50,50.0,80.0,0.0,16.0,24.0,8.0,68.0,211.0,0.0,5.0,100.0,0.0,14280,1440,3360,5040,5,0,0,2,720.0,19080.0,-1
649,86,104,48,31.0,64.0,0.0,18.0,26.0,10.0,41.0,176.0,0.0,4.0,100.0,0.0,1980,0,2580,3300,1,0,0,1,600.0,4560.0,-1
650,89,98,85,25.0,56.0,0.0,19.0,25.0,11.0,35.0,170.0,0.0,0.0,0.0,0.0,480,0,900,2040,0,0,0,0,0.0,1380.0,-1
651,64,86,51,48.0,85.0,0.0,16.0,22.0,10.0,58.0,159.0,0.0,21.0,100.0,0.0,18120,1680,3420,1620,1,0,0,1,420.0,23220.0,-1


In [8]:
model = LocalOutlierFactor()
X = df[measures]
model.fit(X)
y_pred = model.fit_predict(X) 
X[y_pred == -1] # anomalies

,HrAverage,HrMax,HrMin,RmssdAverage,RmssdMax,RmssdMin,RrAverage,RrMax,RrMin,Sdnn_1Average,Sdnn_1Max,Sdnn_1Min,SnoringAverage,SnoringMax,SnoringMin,LightSleepDuration,RemSleepDuration,DeepSleepDuration,WakeUpDuration,WakeUpCount,ManualSleepDuration,UndefinedSleepDuration,OutOfBedCount,OutOfBedTime,TotalSleepTime
0,56,95,47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21180,0,14100,2760,4,0,0,0,0.0,35280.0
84,83,98,62,17.0,36.0,0.0,17.0,22.0,15.0,25.0,39.0,0.0,0.0,0.0,0.0,960,1200,0,720,0,0,0,0,0.0,2160.0
120,90,98,57,26.0,34.0,0.0,19.0,27.0,8.0,32.0,40.0,0.0,0.0,0.0,0.0,5460,0,4980,2220,2,0,0,1,540.0,10440.0
142,70,94,57,23.0,29.0,0.0,15.0,22.0,9.0,46.0,56.0,0.0,0.0,100.0,0.0,20280,4200,1140,3180,1,0,0,1,6960.0,25620.0
194,62,83,47,37.0,53.0,0.0,15.0,31.0,9.0,60.0,97.0,0.0,1.0,100.0,0.0,19800,7200,1200,12060,4,0,0,3,8820.0,28200.0
201,62,81,47,38.0,51.0,0.0,15.0,24.0,10.0,54.0,71.0,0.0,0.0,75.0,0.0,29520,7020,7140,10740,7,0,0,6,0.0,43680.0
221,81,92,70,14.0,28.0,0.0,16.0,23.0,10.0,26.0,56.0,0.0,0.0,0.0,0.0,3060,0,420,6060,2,0,0,1,0.0,3480.0
296,67,86,50,40.0,48.0,0.0,17.0,27.0,10.0,50.0,54.0,0.0,8.0,100.0,0.0,30600,6120,1920,3180,2,0,0,2,5580.0,38640.0
322,65,83,52,50.0,59.0,0.0,16.0,26.0,11.0,58.0,153.0,0.0,0.0,100.0,0.0,21000,7380,1260,2940,2,0,0,2,8460.0,29640.0
328,78,93,50,35.0,42.0,0.0,18.0,30.0,9.0,41.0,51.0,0.0,0.0,0.0,0.0,60,0,60,0,2,0,0,1,420.0,120.0


# utilizzo di XAI

In [9]:
df['StartValue'] = pd.to_datetime(df['StartValue'])
df['EndValue'] = pd.to_datetime(df['EndValue'])

# Trova il primo giorno disponibile nel dataset
start_date = df['EndValue'].min()

# Calcola la differenza in giorni tra ogni data e la data iniziale
df['DaysSinceStart'] = (df['EndValue'] - start_date).dt.days

# Calcola il numero di settimane relative al primo giorno disponibile
df['WeekNumber'] = (df['DaysSinceStart'] // 7) + 1

# Aggiunta del primo e ultimo giorno della settimana come colonne
df['WeekStart'] = df.groupby('WeekNumber')['StartValue'].transform('min')
df['WeekEnd'] = df.groupby('WeekNumber')['EndValue'].transform('max')

# Raggruppa per settimana
weekly_data = df.groupby('WeekNumber').agg({
    'WeekStart': 'first',  # Usa il primo valore della settimana
    'WeekEnd': 'first',    # Usa l'ultimo valore della settimana
    'HrAverage' : ['mean', 'std', 'min', 'max', 'sum'],
    'HrMax' : ['mean', 'std', 'min', 'max', 'sum'],
    'HrMin' : ['mean', 'std', 'min', 'max', 'sum'],
    'RmssdAverage' : ['mean', 'std', 'min', 'max', 'sum'],
    'RmssdMax' : ['mean', 'std', 'min', 'max', 'sum'],
    'RmssdMin' : ['mean', 'std', 'min', 'max', 'sum'],
    'RrAverage' : ['mean', 'std', 'min', 'max', 'sum'],
    'RrMax' : ['mean', 'std', 'min', 'max', 'sum'],
    'RrMin' : ['mean', 'std', 'min', 'max', 'sum'],
    'Sdnn_1Average' : ['mean', 'std', 'min', 'max', 'sum'],
    'Sdnn_1Max' : ['mean', 'std', 'min', 'max', 'sum'],
    'Sdnn_1Min' : ['mean', 'std', 'min', 'max', 'sum'],
    'SnoringAverage' : ['mean', 'std', 'min', 'max', 'sum'],
    'SnoringMax' : ['mean', 'std', 'min', 'max', 'sum'],
    'SnoringMin' : ['mean', 'std', 'min', 'max', 'sum'],
    'LightSleepDuration' : ['mean', 'std', 'min', 'max', 'sum'],
    'RemSleepDuration' : ['mean', 'std', 'min', 'max', 'sum'],
    'DeepSleepDuration' : ['mean', 'std', 'min', 'max', 'sum'],
    'WakeUpDuration' : ['mean', 'std', 'min', 'max', 'sum'],
    'WakeUpCount' : ['mean', 'std', 'min', 'max', 'sum'],
    'ManualSleepDuration' : ['mean', 'std', 'min', 'max', 'sum'],
    'UndefinedSleepDuration' : ['mean', 'std', 'min', 'max', 'sum'],
    'OutOfBedCount' : ['mean', 'std', 'min', 'max', 'sum'],
    'OutOfBedTime' : ['mean', 'std', 'min', 'max', 'sum'],
    'TotalSleepTime': ['mean', 'std', 'min', 'max', 'sum'],
    'Nap': 'sum'  # Conta i pisolini in ogni settimana
}).reset_index()
columnList = ['WeekNumber',
                'WeekStart', 'WeekEnd',
                'MEAN_HrAverage', 'STD_HrAverage', 'MIN_HrAverage', 'MAX_HrAverage', 'SUM_HrAverage',
                'MEAN_HrMax', 'STD_HrMax', 'MIN_HrMax', 'MAX_HrMax', 'SUM_HrMax',
                'MEAN_HrMin', 'STD_HrMin', 'MIN_HrMin', 'MAX_HrMin', 'SUM_HrMin',
                'MEAN_RmssdAverage', 'STD_RmssdAverage', 'MIN_RmssdAverage', 'MAX_RmssdAverage', 'SUM_RmssdAverage',
                'MEAN_RmssdMax','STD_RmssdMax','MIN_RmssdMax','MAX_RmssdMax','SUM_RmssdMax',
                'MEAN_RmssdMin','STD_RmssdMin','MIN_RmssdMin','MAX_RmssdMin','SUM_RmssdMin',
                'MEAN_RrAverage','STD_RrAverage','MIN_RrAverage','MAX_RrAverage','SUM_RrAverage',
                'MEAN_RrMax','STD_RrMax','MIN_RrMax','MAX_RrMax','SUM_RrMax',
                'MEAN_RrMin','STD_RrMin','MIN_RrMin','MAX_RrMin','SUM_RrMin',
                'MEAN_Sdnn_1Average','STD_Sdnn_1Average','MIN_Sdnn_1Average','MAX_Sdnn_1Average','SUM_Sdnn_1Average',
                'MEAN_Sdnn_1Max','STD_Sdnn_1Max','MIN_Sdnn_1Max','MAX_Sdnn_1Max','SUM_Sdnn_1Max',
                'MEAN_Sdnn_1Min','STD_Sdnn_1Min','MIN_Sdnn_1Min','MAX_Sdnn_1Min','SUM_Sdnn_1Min',
                'MEAN_SnoringAverage','STD_SnoringAverage','MIN_SnoringAverage','MAX_SnoringAverage','SUM_SnoringAverage',
                'MEAN_SnoringMax','STD_SnoringMax','MIN_SnoringMax','MAX_SnoringMax','SUM_SnoringMax',
                'MEAN_SnoringMin','STD_SnoringMin','MIN_SnoringMin','MAX_SnoringMin','SUM_SnoringMin',
                'MEAN_LightSleepDuration','STD_LightSleepDuration','MIN_LightSleepDuration','MAX_LightSleepDuration','SUM_LightSleepDuration',
                'MEAN_RemSleepDuration','STD_RemSleepDuration','MIN_RemSleepDuration','MAX_RemSleepDuration','SUM_RemSleepDuration',
                'MEAN_DeepSleepDuration','STD_DeepSleepDuration','MIN_DeepSleepDuration','MAX_DeepSleepDuration','SUM_DeepSleepDuration',
                'MEAN_WakeUpDuration','STD_WakeUpDuration','MIN_WakeUpDuration','MAX_WakeUpDuration','SUM_WakeUpDuration',
                'MEAN_WakeUpCount','STD_WakeUpCount','MIN_WakeUpCount','MAX_WakeUpCount','SUM_WakeUpCount',
                'MEAN_ManualSleepDuration','STD_ManualSleepDuration','MIN_ManualSleepDuration','MAX_ManualSleepDuration','SUM_ManualSleepDuration',
                'MEAN_UndefinedSleepDuration','STD_UndefinedSleepDuration','MIN_UndefinedSleepDuration','MAX_UndefinedSleepDuration','SUM_UndefinedSleepDuration',
                'MEAN_OutOfBedCount','STD_OutOfBedCount','MIN_OutOfBedCount','MAX_OutOfBedCount','SUM_OutOfBedCount',
                'MEAN_OutOfBedTime','STD_OutOfBedTime','MIN_OutOfBedTime','MAX_OutOfBedTime','SUM_OutOfBedTime',
                'MEAN_TotalSleepTime', 'STD_TotalSleepTime', 'MIN_TotalSleepTime', 'MAX_TotalSleepTime', 'SUM_TotalSleepTime',
                'SUM_Nap']
# Rinomina colonne per facilità di utilizzo
weekly_data.columns = columnList

# Aggiunta di una colonna con il range di date della settimana
weekly_data['DateRange'] = weekly_data['WeekStart'].dt.strftime('%d %b %Y') + ' - ' + weekly_data['WeekEnd'].dt.strftime('%d %b %Y')

# Imposta la colonna 'std' a 0 nei casi in cui abbia NaN
columns_with_std = [col for col in weekly_data.columns if 'std' in col.lower()]  # Trova tutte le colonne 'std'

#ci sono casi in cui la standard deviation non si pò fare perchè la persona ha dormito solo 1 volta nella settimana, quindi metto a 0 la std
for col in columns_with_std:
    weekly_data[col] = weekly_data[col].fillna(0)  # Riempie i NaN solo nelle colonne 'std'

In [10]:
model = IsolationForest()
# Rimuove WeekStart e WeekEnd usando una list comprehension
columnsToModel = [col for col in columnList if col not in ['WeekNumber','WeekStart', 'WeekEnd', 'DateRange']]
X = weekly_data[columnsToModel]
X = X.astype('float64')
model.fit(X)

IsolationForest()

In [11]:
# Predizione delle anomalie
weekly_data['anomaly_score'] = model.decision_function(X)  # Score di anomalia
weekly_data['is_anomaly'] = model.predict(X)  # -1: anomalia, 1: normale

# Converte i valori -1 e 1 in True/False per facilità
weekly_data['is_anomaly'] = weekly_data['is_anomaly'] == -1

In [12]:
weekly_data[weekly_data['is_anomaly']==True] # anomalies

,WeekNumber,WeekStart,WeekEnd,MEAN_HrAverage,STD_HrAverage,MIN_HrAverage,MAX_HrAverage,SUM_HrAverage,MEAN_HrMax,STD_HrMax,MIN_HrMax,MAX_HrMax,SUM_HrMax,MEAN_HrMin,STD_HrMin,MIN_HrMin,MAX_HrMin,SUM_HrMin,MEAN_RmssdAverage,STD_RmssdAverage,MIN_RmssdAverage,MAX_RmssdAverage,SUM_RmssdAverage,MEAN_RmssdMax,STD_RmssdMax,MIN_RmssdMax,MAX_RmssdMax,SUM_RmssdMax,MEAN_RmssdMin,STD_RmssdMin,MIN_RmssdMin,MAX_RmssdMin,SUM_RmssdMin,MEAN_RrAverage,STD_RrAverage,MIN_RrAverage,MAX_RrAverage,SUM_RrAverage,MEAN_RrMax,STD_RrMax,MIN_RrMax,MAX_RrMax,SUM_RrMax,MEAN_RrMin,STD_RrMin,MIN_RrMin,MAX_RrMin,SUM_RrMin,MEAN_Sdnn_1Average,STD_Sdnn_1Average,MIN_Sdnn_1Average,MAX_Sdnn_1Average,SUM_Sdnn_1Average,MEAN_Sdnn_1Max,STD_Sdnn_1Max,MIN_Sdnn_1Max,MAX_Sdnn_1Max,SUM_Sdnn_1Max,MEAN_Sdnn_1Min,STD_Sdnn_1Min,MIN_Sdnn_1Min,MAX_Sdnn_1Min,SUM_Sdnn_1Min,MEAN_SnoringAverage,STD_SnoringAverage,MIN_SnoringAverage,MAX_SnoringAverage,SUM_SnoringAverage,MEAN_SnoringMax,STD_SnoringMax,MIN_SnoringMax,MAX_SnoringMax,SUM_SnoringMax,MEAN_SnoringMin,STD_SnoringMin,MIN_SnoringMin,MAX_SnoringMin,SUM_SnoringMin,MEAN_LightSleepDuration,STD_LightSleepDuration,MIN_LightSleepDuration,MAX_LightSleepDuration,SUM_LightSleepDuration,MEAN_RemSleepDuration,STD_RemSleepDuration,MIN_RemSleepDuration,MAX_RemSleepDuration,SUM_RemSleepDuration,MEAN_DeepSleepDuration,STD_DeepSleepDuration,MIN_DeepSleepDuration,MAX_DeepSleepDuration,SUM_DeepSleepDuration,MEAN_WakeUpDuration,STD_WakeUpDuration,MIN_WakeUpDuration,MAX_WakeUpDuration,SUM_WakeUpDuration,MEAN_WakeUpCount,STD_WakeUpCount,MIN_WakeUpCount,MAX_WakeUpCount,SUM_WakeUpCount,MEAN_ManualSleepDuration,STD_ManualSleepDuration,MIN_ManualSleepDuration,MAX_ManualSleepDuration,SUM_ManualSleepDuration,MEAN_UndefinedSleepDuration,STD_UndefinedSleepDuration,MIN_UndefinedSleepDuration,MAX_UndefinedSleepDuration,SUM_UndefinedSleepDuration,MEAN_OutOfBedCount,STD_OutOfBedCount,MIN_OutOfBedCount,MAX_OutOfBedCount,SUM_OutOfBedCount,MEAN_OutOfBedTime,STD_OutOfBedTime,MIN_OutOfBedTime,MAX_OutOfBedTime,SUM_OutOfBedTime,MEAN_TotalSleepTime,STD_TotalSleepTime,MIN_TotalSleepTime,MAX_TotalSleepTime,SUM_TotalSleepTime,SUM_Nap,DateRange,anomaly_score,is_anomaly
0,1,2023-08-05 21:42:00,2023-08-06 08:16:00,56.000000,0.000000,56,56,56,95.000000,0.000000,95,95,95,47.000000,0.000000,47,47,47,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21180.000000,0.000000,21180,21180,21180,0.000000,0.000000,0,0,0,14100.000000,0.000000,14100,14100,14100,2760.000000,0.000000,2760,2760,2760,4.000000,0.000000,4,4,4,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0.0,0.0,0.0,35280.000000,0.000000,35280.0,35280.0,35280.0,False,05 Aug 2023 - 06 Aug 2023,-0.213581,True
5,8,2023-09-24 23:35:00,2023-09-30 13:38:00,68.285714,7.296444,60,83,478,86.714286,4.608481,80,93,607,51.428571,3.866831,45,58,360,31.000000,13.366625,2.0,42.0,217.0,41.285714,13.548853,12.0,51.0,289.0,4.571429,12.094863,0.0,32.0,32.0,16.285714,1.253566,15.0,19.0,114.0,24.285714,1.603567,22.0,27.0,170.0,9.000000,1.000000,8.0,11.0,63.0,41.142857,14.530756,9.0,53.0,288.0,51.000000,9.660918,32.0,64.0,357.0,6.000000,15.874508,0.0,42.0,42.0,4.285714,3.638419,0.0,9.0,30.0,85.714286,37.796447,0.0,100.0,600.0,0.0,0.0,0.0,0.0,0.0,17682.857143,8039.680164,660,24180,123780,3274.285714,1867.679997,0,5220,22920,3102.857143,2191.907193,0,6780,21720,5562.857143,2807.357680,2040,10680,38940,2.000000,1.825742,0,5,14,0.0,0.0,0,0,0,0.0,0.0,0,0,0,1.285714,1.112697,0,3,9,1671.428571,2649.851748,0.0,7560.0,11700.0,24060.000000,10699.476623,660.0,33360.0,168420.0,1,24 Sep 2023 - 30 Sep 2023,-0.012180,True
47,51,2024-07-20 23:42:00,2024-07-21 08:16:00,62.000000,0.000000,62,62,62,78.000000,0.000000,78,78,78,49.000000,0.000000,49,49,49

In [13]:
# --- Spiegazione con SHAP ---
# Usa SHAP per spiegare il modello Isolation Forest
explainer = shap.Explainer(model, X)
shap_values = explainer(X)

In [14]:
# --- Visualizzazione con Plotly ---
# Scatter interattivo per il punteggio di anomalia
fig = px.scatter(
    weekly_data,
    x="WeekNumber",
    y="anomaly_score",
    color="is_anomaly",  # La variabile di colore per evidenziare le anomalie
    title="Punteggi di Anomalia per Settimana",
    labels={"is_anomaly": "Anomalia"},
    hover_data=['DateRange'],  # Mostra le informazioni delle colonne in columnList al passaggio del mouse (NON va probabilmente ci sono troppe colonne)
    color_discrete_map={True: "red", False: "blue"}  # Impostazione dei colori: rosso per anomalia, blu per normale
)

# Aggiornamento dell'aspetto del grafico
fig.update_layout(
    xaxis_title="Numero della Settimana",
    yaxis_title="Punteggio di Anomalia",
)

# Visualizza il grafico
fig.show()

In [15]:
# SHAP values per feature (grafico a barre per l'importanza delle feature)
shap_summary = shap_values.values.mean(axis=0)
feature_importance = pd.DataFrame({
    "Feature": X.columns,
    "SHAP Value": shap_summary
}).sort_values(by="SHAP Value", ascending=False)

fig_shap = px.bar(
    feature_importance,
    x="SHAP Value",
    y="Feature",
    orientation="h",
    title="Importanza delle Feature (SHAP)",
    labels={"SHAP Value": "Valore SHAP", "Feature": "Feature"},
)
fig_shap.update_layout(yaxis=dict(categoryorder="total ascending"))
fig_shap.show()